In [ ]:
# for speech-to-text
import speech_recognition as sr
# for text-to-speech
import pyttsx3
# for language model
import transformers
import os
import time
# for data
import os
import datetime
import numpy as np
import pywhatkit
import wikipedia

In [ ]:
# Building the AI
class ChatBot():
    def __init__(self, name):
        print("----- Starting up", name, "-----")
        self.name = name
    def listen(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            print("Listening...")
            audio = recognizer.listen(mic)
            self.text="ERROR"
        try:
            self.text = recognizer.recognize_google(audio)
            print("Me  --> ", self.text)
        except:
            print("Me  -->  ERROR")
    @staticmethod
    def talk(text):
        print("Dev --> ", text)
        engine = pyttsx3.init('sapi5')
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[3].id)
        engine.setProperty('rate',200)
        engine.say(text)
        engine.runAndWait()
    def wake_up(self, text):
        return True if self.name in text.lower() else False
    @staticmethod
    def action_time():
        return datetime.datetime.now().time().strftime('%H:%M')

In [ ]:
# Running the AI
if __name__ == "__main__":
    ai = ChatBot(name="bob")
    nlp = transformers.pipeline("conversational", model="microsoft/DialoGPT-medium")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex=True
    while ex:
        ai.listen()
        ## wake up
        if ai.wake_up(ai.text) is True:
            res = "Hello I am Bob the AI, what can I do for you?"
        ## action time
        elif "time" in ai.text:
            res = ai.action_time()
        ## respond politely
        elif any(i in ai.text for i in ["thank","thanks"]):
            res = np.random.choice(["you're welcome!","anytime!","no problem!","cool!","I'm here if you need me!","mention not"])
        elif any(i in ai.text for i in ["exit","close"]):
            res = np.random.choice(["Tata","Have a good day","Bye","Goodbye","Hope to meet soon","peace out!"])
            ex=False
        elif "play" in ai.text:
            song = ai.text.replace('play','')
            res = np.random.choice(["Here is the song", "Playing"])
            pywhatkit.playonyt(song)
        elif any(i in ai.text for i in ["tell me about", "summerize", "explain", "who is"]):
            res = wikipedia.summary(ai.text)
        ## conversation
        else:   
            if ai.text=="ERROR":
                res="Sorry, come again?"
            else:
                chat = nlp(transformers.Conversation(ai.text), pad_token_id=50256)
                res = str(chat)
                res = res[res.find("bot >> ")+6:].strip()
        ai.talk(res)
    print("----- Closing down Bob -----")